In [1]:
import os
import numpy as np
import random
import cv2
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, cohen_kappa_score)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import (VGG16, ResNet50, ResNet50V2, MobileNet,
                                           MobileNetV2, DenseNet121, EfficientNetB0,
                                           EfficientNetV2B0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Flatten, GlobalAveragePooling2D,
                                     Dropout)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

2024-10-01 13:19:48.633047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 13:19:48.874187: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 13:19:48.962230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 13:19:49.609365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-01 13:19:53.079983: W tensorflow/compiler/tf2

In [2]:
def load_data(polyp_folder, non_polyp_folder, image_size=(256, 256)):
    images = []
    labels = []
    
    # Load polyp images
    for filename in os.listdir(polyp_folder):
        if filename.lower().endswith(('.jpg', '.png')):
            image = cv2.imread(os.path.join(polyp_folder, filename))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(1)
    
    # Load non-polyp images
    for filename in os.listdir(non_polyp_folder):
        if filename.lower().endswith(('.jpg', '.png')):
            image = cv2.imread(os.path.join(non_polyp_folder, filename))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(0)
    
    images = np.array(images)
    labels = np.array(labels)
    
    # Encode labels
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels)
    
    # Shuffle data
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    images = images[indices]
    labels = labels[indices]
    
    print(f"\nTotal number of images: {images.shape[0]}, Image shape: {images.shape[1:]}")
    
    return images, labels

In [3]:
def create_model(model_name, input_shape=(256, 256, 3), num_classes=2):
    base_model = None
    
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'ResNet50V2':
        base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'DenseNet':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'EfficientNetV2B0':
        base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Model name not recognized.")
    
    base_model.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [4]:
from sklearn.metrics import confusion_matrix

def evaluate_model(X, y, model_name):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [],
               'f1_score': [], 'auc': [], 'kappa': []}
    
    fold = 1
    for train_idx, val_idx in kfold.split(X):
        print(f"\nTraining fold {fold} for {model_name}...")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        model = create_model(model_name)

        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                     rotation_range=20, zoom_range=0.2)
        datagen.fit(X_train)

        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50, validation_data=(X_val, y_val), verbose=1)

        y_pred_prob = model.predict(X_val)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_true = np.argmax(y_val, axis=1)

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_prob[:,1])
        kappa = cohen_kappa_score(y_true, y_pred)

        metrics['accuracy'].append(acc)
        metrics['precision'].append(prec)
        metrics['recall'].append(rec)
        metrics['f1_score'].append(f1)
        metrics['auc'].append(auc)
        metrics['kappa'].append(kappa)
        
        print(f"Fold {fold} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
              f"Recall: {rec:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}, Kappa: {kappa:.4f}")
        
        fold += 1
    return metrics

In [22]:
model_names = ['VGG16', 'ResNet50', 'ResNet50V2', 'MobileNet',
               'MobileNetV2', 'DenseNet', 'EfficientNetB0', 'EfficientNetV2B0']

all_metrics = {}

polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

for model_name in model_names:
    metrics = evaluate_model(X, y, model_name)
    all_metrics[model_name] = metrics


Total number of images: 1070, Image shape: (256, 256, 3)

Training fold 1 for EfficientNetB0...
Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.5428 - loss: 0.7344 - val_accuracy: 0.6075 - val_loss: 0.6600
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 317ms/step - accuracy: 0.5649 - loss: 0.7047 - val_accuracy: 0.6729 - val_loss: 0.6158
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - accuracy: 0.6510 - loss: 0.6328 - val_accuracy: 0.7056 - val_loss: 0.5810
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.6786 - loss: 0.6163 - val_accuracy: 0.7336 - val_loss: 0.5531
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 316ms/step - accuracy: 0.6727 - loss: 0.5913 - val_accuracy: 0.7477 - val_loss: 0.5308
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 317ms/step - accuracy: 0.7347 - loss: 0.5560 - val_accuracy: 0.7430 - val_loss: 0.5118
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 316ms/step - accuracy: 0.7303 - loss: 0.5527 - val_accuracy: 0.7383 - val_loss: 0.4953
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 315ms/step - accuracy: 0.7459 - loss: 0.5383 - val_accuracy: 0.7757 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 48s 985ms/step - accuracy: 0.5348 - loss: 0.7723 - val_accuracy: 0.6121 - val_loss: 0.6474
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 333ms/step - accuracy: 0.6151 - loss: 0.6876 - val_accuracy: 0.7430 - val_loss: 0.5913
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 13s 346ms/step - accuracy: 0.5889 - loss: 0.6809 - val_accuracy: 0.7710 - val_loss: 0.5541
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 326ms/step - accuracy: 0.6270 - loss: 0.6396 - val_accuracy: 0.7897 - val_loss: 0.5226
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.6397 - loss: 0.6263 - val_accuracy: 0.8224 - val_loss: 0.4958
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 308ms/step - accuracy: 0.7271 - loss: 0.5516 - val_accuracy: 0.8224 - val_loss: 0.4751
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.6941 - loss: 0.5775 - val_accuracy: 0.8364 - val_loss: 0.4565
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 310ms/step - accuracy: 0.7703 - loss: 0.5069 - val_accuracy: 0.845

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 49s 986ms/step - accuracy: 0.4684 - loss: 0.8391 - val_accuracy: 0.5140 - val_loss: 0.7115
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.5396 - loss: 0.7366 - val_accuracy: 0.5841 - val_loss: 0.6562
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.5830 - loss: 0.7091 - val_accuracy: 0.6449 - val_loss: 0.6156
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.6127 - loss: 0.6451 - val_accuracy: 0.7009 - val_loss: 0.5864
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 312ms/step - accuracy: 0.6186 - loss: 0.6603 - val_accuracy: 0.7430 - val_loss: 0.5606
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 313ms/step - accuracy: 0.6776 - loss: 0.6021 - val_accuracy: 0.7523 - val_loss: 0.5446
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 312ms/step - accuracy: 0.7249 - loss: 0.5583 - val_accuracy: 0.7757 - val_loss: 0.5273
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 311ms/step - accuracy: 0.7249 - loss: 0.5452 - val_accuracy: 0.780

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 48s 978ms/step - accuracy: 0.5483 - loss: 0.7223 - val_accuracy: 0.6682 - val_loss: 0.6033
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.6026 - loss: 0.6711 - val_accuracy: 0.6963 - val_loss: 0.5727
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 308ms/step - accuracy: 0.6030 - loss: 0.6622 - val_accuracy: 0.7290 - val_loss: 0.5498
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.6631 - loss: 0.5970 - val_accuracy: 0.7243 - val_loss: 0.5318
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.6967 - loss: 0.5804 - val_accuracy: 0.7290 - val_loss: 0.5144
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 312ms/step - accuracy: 0.7284 - loss: 0.5565 - val_accuracy: 0.7523 - val_loss: 0.5014
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 310ms/step - accuracy: 0.7323 - loss: 0.5531 - val_accuracy: 0.7664 - val_loss: 0.4899
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 312ms/step - accuracy: 0.7739 - loss: 0.4992 - val_accuracy: 0.775

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5458 - loss: 0.8017 - val_accuracy: 0.5280 - val_loss: 0.7278
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 13s 303ms/step - accuracy: 0.5581 - loss: 0.7253 - val_accuracy: 0.6262 - val_loss: 0.6645
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 318ms/step - accuracy: 0.6045 - loss: 0.6880 - val_accuracy: 0.6869 - val_loss: 0.6184
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.6362 - loss: 0.6390 - val_accuracy: 0.7196 - val_loss: 0.5842
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.6315 - loss: 0.6516 - val_accuracy: 0.7710 - val_loss: 0.5534
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.6854 - loss: 0.6034 - val_accuracy: 0.7991 - val_loss: 0.5278
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.7385 - loss: 0.5784 - val_accuracy: 0.8271 - val_loss: 0.5071
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.7007 - loss: 0.5604 - val_accuracy: 0.8271 -

In [6]:
for model_name, metrics in all_metrics.items():
    print(f"\nModel: {model_name}")
    print(f"Average Accuracy: {np.mean(metrics['accuracy']):.4f}")
    print(f"Average Precision: {np.mean(metrics['precision']):.4f}")
    print(f"Average Recall: {np.mean(metrics['recall']):.4f}")
    print(f"Average F1 Score: {np.mean(metrics['f1_score']):.4f}")
    print(f"Average AUC: {np.mean(metrics['auc']):.4f}")
    print(f"Average Kappa: {np.mean(metrics['kappa']):.4f}")



Model: VGG16
Average Accuracy: 0.8224
Average Precision: 0.8269
Average Recall: 0.8267
Average F1 Score: 0.8260
Average AUC: 0.9028
Average Kappa: 0.6420

Model: ResNet50
Average Accuracy: 0.8570
Average Precision: 0.8607
Average Recall: 0.8619
Average F1 Score: 0.8606
Average AUC: 0.9219
Average Kappa: 0.7112

Model: ResNet50V2
Average Accuracy: 0.7776
Average Precision: 0.7823
Average Recall: 0.7856
Average F1 Score: 0.7831
Average AUC: 0.8627
Average Kappa: 0.5497

Model: MobileNet
Average Accuracy: 0.6907
Average Precision: 0.6887
Average Recall: 0.7437
Average F1 Score: 0.7111
Average AUC: 0.7597
Average Kappa: 0.3825

Model: MobileNetV2
Average Accuracy: 0.7430
Average Precision: 0.7462
Average Recall: 0.7681
Average F1 Score: 0.7528
Average AUC: 0.8045
Average Kappa: 0.4877

Model: DenseNet
Average Accuracy: 0.7215
Average Precision: 0.7133
Average Recall: 0.7729
Average F1 Score: 0.7381
Average AUC: 0.8045
Average Kappa: 0.4420

Model: EfficientNetB0
Average Accuracy: 0.8598
A

## Attention Mechanism

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Dense, Flatten, GlobalAveragePooling2D,
                                     GlobalMaxPooling2D, Dropout, Conv2D, Reshape, multiply,
                                     Permute, Activation, Lambda, Concatenate, Add, Multiply)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import (ResNet50, ResNet50V2, VGG16, MobileNet,
                                           MobileNetV2, DenseNet121, EfficientNetB0,
                                           EfficientNetV2B0)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import numpy as np
import os
import cv2
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, cohen_kappa_score)
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Reshape, Activation, Lambda, Add


In [5]:
def spatial_attention_block(input_tensor):
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(input_tensor)
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(input_tensor)
    concat = Concatenate(axis=3)([avg_pool, max_pool])

    spatial_attention = Conv2D(filters=1,
                               kernel_size=7,
                               strides=1,
                               padding='same',
                               activation='sigmoid',
                               kernel_initializer='he_normal',
                               use_bias=False)(concat)

    output = multiply([input_tensor, spatial_attention])
    return output

def squeeze_excite_block(input_tensor, ratio=16):
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape((1, 1, filters))(se)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    x = multiply([input_tensor, se])
    return x


def spatial_attention_block(input_tensor):
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(input_tensor)
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(input_tensor)
    concat = Concatenate(axis=3)([avg_pool, max_pool])

    spatial_attention = Conv2D(filters=1,
                               kernel_size=7,
                               strides=1,
                               padding='same',
                               activation='sigmoid',
                               kernel_initializer='he_normal',
                               use_bias=False)(concat)

    output = multiply([input_tensor, spatial_attention])
    return output

def non_local_block(input_tensor, compression=2):
   
    H, W, C = input_tensor.shape[1], input_tensor.shape[2], input_tensor.shape[3]

    if None in [H, W, C]:
        H = tf.shape(input_tensor)[1]
        W = tf.shape(input_tensor)[2]
        C = tf.shape(input_tensor)[3]

    inter_channels = max(1, C // compression)

    theta = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(input_tensor)
    theta = Reshape((-1, inter_channels))(theta)  # Shape: (batch_size, H*W, inter_channels)

    phi = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(input_tensor)
    phi = Reshape((-1, inter_channels))(phi)      # Shape: (batch_size, H*W, inter_channels)
    phi = Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(phi)  # Shape: (batch_size, inter_channels, H*W)

    f = Lambda(lambda x: tf.matmul(x[0], x[1]))([theta, phi])  # Shape: (batch_size, H*W, H*W)
    f = Activation('softmax')(f)

    g = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(input_tensor)
    g = Reshape((-1, inter_channels))(g)  # Shape: (batch_size, H*W, inter_channels)

    y = Lambda(lambda x: tf.matmul(x[0], x[1]))([f, g])  # Shape: (batch_size, H*W, inter_channels)
    y = Reshape((H, W, inter_channels))(y)               # Shape: (batch_size, H, W, inter_channels)
    y = Conv2D(C, kernel_size=1, strides=1, padding='same')(y)

    output = Add()([input_tensor, y])
    return output


def gc_block(input_tensor, ratio=16):
  
    C = input_tensor.shape[-1]
    if C is None:
        C = tf.shape(input_tensor)[-1]

    x = input_tensor
    
    context = Conv2D(1, kernel_size=1)(x)
    context = Reshape((-1, 1))(context)
    context = Activation('softmax')(context)

    x_flat = Reshape((-1, C))(x)

    context_vector = Lambda(lambda x: tf.matmul(x[0], x[1], transpose_a=True))([context, x_flat])
    context_vector = Reshape((1, 1, C))(context_vector)

    transform = Conv2D(C // ratio, kernel_size=1, activation='relu')(context_vector)
    transform = Conv2D(C, kernel_size=1)(transform)

    output = Add()([input_tensor, transform])

    return output


def self_attention_block(input_tensor):
    
    H, W, C = input_tensor.shape[1], input_tensor.shape[2], input_tensor.shape[3]

    if None in [H, W, C]:
        H = tf.shape(input_tensor)[1]
        W = tf.shape(input_tensor)[2]
        C = tf.shape(input_tensor)[3]

    f = Conv2D(C // 8, kernel_size=1, strides=1, padding='same')(input_tensor)
    g = Conv2D(C // 8, kernel_size=1, strides=1, padding='same')(input_tensor)
    h = Conv2D(C // 2, kernel_size=1, strides=1, padding='same')(input_tensor)

    f_flat = Reshape((-1, C // 8))(f)
    g_flat = Reshape((-1, C // 8))(g)
    h_flat = Reshape((-1, C // 2))(h)

    s = Lambda(lambda x: tf.matmul(x[0], x[1], transpose_b=True))([f_flat, g_flat])
    beta = Activation('softmax')(s)

    o = Lambda(lambda x: tf.matmul(x[0], x[1]))([beta, h_flat])
    o = Reshape((H, W, C // 2))(o)
    o = Conv2D(C, kernel_size=1, strides=1, padding='same')(o)

    output = Add()([input_tensor, o])
    return output

def gated_attention_block(input_tensor):
 
    channels = K.int_shape(input_tensor)[-1]
    gating_signal = Conv2D(channels, kernel_size=1, activation='sigmoid')(input_tensor)
    x = Multiply()([input_tensor, gating_signal])
    return x


In [6]:
def create_model_with_attention(model_name, attention_type='se', input_shape=(256, 256, 3), num_classes=2):
    """Create a model with a specified attention mechanism.

    Args:
        model_name: Name of the base pre-trained model.
        attention_type: Type of attention mechanism to apply.
        input_shape: Shape of input images.
        num_classes: Number of output classes.

    Returns:
        A compiled Keras model.
    """
    # Load base model
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'ResNet50V2':
        base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'MobileNet':
        base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'DenseNet':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'EfficientNetV2B0':
        base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Model name not recognized.")

    # Freeze base model
    base_model.trainable = False

    # Base model output
    x = base_model.output

    # Apply attention mechanism
    if attention_type == 'spatial':
        x = spatial_attention_block(x)
    elif attention_type == 'se':
        x = squeeze_excite_block(x)
    elif attention_type == 'non_local':
        x = non_local_block(x)
    elif attention_type == 'gc':
        x = gc_block(x)
    elif attention_type == 'self_attention':
        x = self_attention_block(x)
    elif attention_type == 'gated_attention':
        x = gated_attention_block(x)
    else:
        raise ValueError(f"Unknown attention type: {attention_type}")

    # Add classification head
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=base_model.input, outputs=outputs)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
def evaluate_model_with_attention(X, y, model_name, attention_type):
    """Evaluate a model with a specified attention mechanism using cross-validation.

    Args:
        X: Input data.
        y: Labels.
        model_name: Name of the base pre-trained model.
        attention_type: Type of attention mechanism.

    Returns:
        Dictionary of performance metrics.
    """
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [],
               'f1_score': [], 'auc': [], 'kappa': []}

    fold = 1
    for train_idx, val_idx in kfold.split(X):
        print(f"\nTraining fold {fold} with attention: {attention_type}")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Create model
        model = create_model_with_attention(model_name, attention_type)

        # Data augmentation
        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                     rotation_range=20, zoom_range=0.2)
        datagen.fit(X_train)

        # Train model
        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50, validation_data=(X_val, y_val), verbose=1)

        # Predictions
        y_pred_prob = model.predict(X_val)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_true = np.argmax(y_val, axis=1)

        # Compute metrics
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_prob[:, 1])
        kappa = cohen_kappa_score(y_true, y_pred)

        # Append metrics
        metrics['accuracy'].append(acc)
        metrics['precision'].append(prec)
        metrics['recall'].append(rec)
        metrics['f1_score'].append(f1)
        metrics['auc'].append(auc)
        metrics['kappa'].append(kappa)

        print(f"Fold {fold} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
              f"Recall: {rec:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}, Kappa: {kappa:.4f}")

        fold += 1
    return metrics


In [6]:
def evaluate_model_with_attention(X, y, model_name, attention_type):
    """Evaluate a model with a specified attention mechanism using cross-validation.

    Args:
        X: Input data.
        y: Labels.
        model_name: Name of the base pre-trained model.
        attention_type: Type of attention mechanism.

    Returns:
        Dictionary of performance metrics.
    """
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [],
               'f1_score': [], 'auc': [], 'kappa': []}

    fold = 1
    for train_idx, val_idx in kfold.split(X):
        print(f"\nTraining fold {fold} with attention: {attention_type}")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Create model
        model = create_model_with_attention(model_name, attention_type)

        # Data augmentation
        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                     rotation_range=20, zoom_range=0.2)
        datagen.fit(X_train)

        # Train model
        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50, validation_data=(X_val, y_val), verbose=1)

        # Predictions
        y_pred_prob = model.predict(X_val)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_true = np.argmax(y_val, axis=1)

        # Compute metrics
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_prob[:, 1])
        kappa = cohen_kappa_score(y_true, y_pred)

        # Append metrics
        metrics['accuracy'].append(acc)
        metrics['precision'].append(prec)
        metrics['recall'].append(rec)
        metrics['f1_score'].append(f1)
        metrics['auc'].append(auc)
        metrics['kappa'].append(kappa)

        print(f"Fold {fold} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
              f"Recall: {rec:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}, Kappa: {kappa:.4f}")

        fold += 1

    # Calculate mean and standard deviation for each metric
    summary_metrics = {}
    for metric in metrics:
        mean_val = np.mean(metrics[metric])
        std_val = np.std(metrics[metric])
        summary_metrics[metric] = f"{mean_val:.4f} ± {std_val:.4f}"
        print(f"{metric.capitalize()} - Mean: {mean_val:.4f}, Std: {std_val:.4f}")

    return summary_metrics


polyp_folder = '../main_code_previous/Training_1/abnomral_training_images'
non_polyp_folder = '../main_code_previous/Training_1/normal_training_images'
X_train, y_train = load_data(polyp_folder, non_polyp_folder)

polyp_folder_testing = '../main_code_previous/Testing_1/abnormal_testing_images'
non_polyp_folder_testing = '../main_code_previous/Testing_1/normal_testing_images'
X_test, y_test = load_data(polyp_folder_testing, non_polyp_folder_testing)


Total number of images: 820, Image shape: (256, 256, 3)

Total number of images: 246, Image shape: (256, 256, 3)


In [10]:
def train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, model_name, attention_type):
    """Train and evaluate a model with a specified attention mechanism.

    Args:
        X_train: Training data.
        y_train: Training labels.
        X_test: Testing data.
        y_test: Testing labels.
        model_name: Name of the base pre-trained model.
        attention_type: Type of attention mechanism.

    Returns:
        Dictionary of performance metrics.
    """
    # Create a dictionary to store metrics
    metrics = {}

    print(f"\nTraining model with attention: {attention_type}")

    # Create model
    model = create_model_with_attention(model_name, attention_type)

    # Data augmentation
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                 rotation_range=20, zoom_range=0.2)
    datagen.fit(X_train)

    # Train model
    model.fit(datagen.flow(X_train, y_train, batch_size=32),
              epochs=50, validation_data=(X_test, y_test), verbose=1)

    # Predictions
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_prob[:, 1])
    kappa = cohen_kappa_score(y_true, y_pred)

    # Store metrics
    metrics['accuracy'] = acc
    metrics['precision'] = prec
    metrics['recall'] = rec
    metrics['f1_score'] = f1
    metrics['auc'] = auc
    metrics['kappa'] = kappa

    print(f"\nResults - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
          f"Recall: {rec:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}, Kappa: {kappa:.4f}")

    return metrics

polyp_folder = '../main_code_previous/Training_1/abnomral_training_images'
non_polyp_folder = '../main_code_previous/Training_1/normal_training_images'
X_train, y_train = load_data(polyp_folder, non_polyp_folder)

polyp_folder_testing = '../main_code_previous/Testing_1/abnormal_testing_images'
non_polyp_folder_testing = '../main_code_previous/Testing_1/normal_testing_images'
X_test, y_test = load_data(polyp_folder_testing, non_polyp_folder_testing)


Total number of images: 820, Image shape: (256, 256, 3)

Total number of images: 246, Image shape: (256, 256, 3)


In [7]:
def train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, model_name, attention_type, runs=5):
    """Train and evaluate a model with a specified attention mechanism over multiple runs.

    Args:
        X_train: Training data.
        y_train: Training labels.
        X_test: Testing data.
        y_test: Testing labels.
        model_name: Name of the base pre-trained model.
        attention_type: Type of attention mechanism.
        runs: Number of runs to average over for calculating mean and standard deviation.

    Returns:
        Dictionary of performance metrics with mean and std.
    """
    # Create a dictionary to store metrics over multiple runs
    all_metrics = {'accuracy': [], 'precision': [], 'recall': [],
                   'f1_score': [], 'auc': [], 'kappa': []}

    for run in range(1, runs + 1):
        print(f"\nTraining run {run} with attention: {attention_type}")

        # Create model
        model = create_model_with_attention(model_name, attention_type)

        # Data augmentation
        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                     rotation_range=20, zoom_range=0.2)
        datagen.fit(X_train)

        # Train model
        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50, validation_data=(X_test, y_test), verbose=1)

        # Predictions
        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_true = np.argmax(y_test, axis=1)

        # Compute metrics
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_prob[:, 1])
        kappa = cohen_kappa_score(y_true, y_pred)

        # Append metrics
        all_metrics['accuracy'].append(acc)
        all_metrics['precision'].append(prec)
        all_metrics['recall'].append(rec)
        all_metrics['f1_score'].append(f1)
        all_metrics['auc'].append(auc)
        all_metrics['kappa'].append(kappa)

        print(f"Run {run} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
              f"Recall: {rec:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}, Kappa: {kappa:.4f}")

    # Calculate mean and standard deviation for each metric
    summary_metrics = {}
    for metric in all_metrics:
        mean_val = np.mean(all_metrics[metric])
        std_val = np.std(all_metrics[metric])
        summary_metrics[metric] = f"{mean_val:.4f} ± {std_val:.4f}"
        print(f"{metric.capitalize()} - Mean: {mean_val:.4f}, Std: {std_val:.4f}")

    return summary_metrics


# Load data
polyp_folder = '../main_code_previous/Training_1/abnomral_training_images'
non_polyp_folder = '../main_code_previous/Training_1/normal_training_images'
X_train, y_train = load_data(polyp_folder, non_polyp_folder)

polyp_folder_testing = '../main_code_previous/Testing_1/abnormal_testing_images'
non_polyp_folder_testing = '../main_code_previous/Testing_1/normal_testing_images'
X_test, y_test = load_data(polyp_folder_testing, non_polyp_folder_testing)

# Evaluate model
# metrics_summary = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, model_name='YourModel', attention_type='YourAttentionType', runs=5)



Total number of images: 820, Image shape: (256, 256, 3)

Total number of images: 246, Image shape: (256, 256, 3)


In [9]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['spatial', 'se','gc'] # 'gated_attention', 'non_local', 'self_attention'
attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics



Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with spatial attention...

Training run 1 with attention: spatial


2024-10-01 00:54:35.840936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 613 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:c1:00.0, compute capability: 7.5


Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727744092.747032 1443267 service.cc:146] XLA service 0x7fdf9404d480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727744092.747084 1443267 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-10-01 00:54:53.145228: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-01 00:54:54.936338: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN versio

 1/26 ━━━━━━━━━━━━━━━━━━━━ 11:25 27s/step - accuracy: 0.4500 - loss: 0.6765

...

26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5472 - loss: 0.6856 - val_accuracy: 0.7520 - val_loss: 0.6164
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - accuracy: 0.6049 - loss: 0.6647 - val_accuracy: 0.7967 - val_loss: 0.5846
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.6778 - loss: 0.6144 - val_accuracy: 0.8252 - val_loss: 0.5443
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.7256 - loss: 0.5798 - val_accuracy: 0.8171 - val_loss: 0.5038
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.7821 - loss: 0.5329 - val_accuracy: 0.8252 - val_loss: 0.4694
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8264 - loss: 0.4880 - val_accuracy: 0.8455 - val_loss: 0.4365
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8226 - loss: 0.4598 - val_accuracy: 0.8211 - val_loss: 0.4191
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.7975 - loss: 0.4529 - val_accuracy: 0.8496 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5179 - loss: 0.7149 - val_accuracy: 0.5000 - val_loss: 0.6894
Epoch 2/50


2024-10-01 02:13:52.551606: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 387.03MiB (rounded to 405832192)requested by op 
2024-10-01 02:13:52.552076: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-10-01 02:13:52.552092: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 17713, Chunks in use: 17713. 4.32MiB allocated for chunks. 4.32MiB in use in bin. 112.8KiB client-requested in use in bin.
2024-10-01 02:13:52.552103: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 621, Chunks in use: 621. 386.8KiB allocated for chunks. 386.8KiB in use in bin. 313.8KiB client-requested in use in bin.
2024-10-01 02:13:52.552112: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (1024): 	Total Chunks: 243, Chunks in use: 243. 272.2KiB allocated for chunks. 272.2KiB in use in bin. 242.9KiB client-requested in

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/users/syellu1/.conda/

In [10]:
attention_results

{'spatial': {'accuracy': '0.8789 ± 0.0079',
  'precision': '0.9195 ± 0.0055',
  'recall': '0.8559 ± 0.0136',
  'f1_score': '0.8865 ± 0.0080',
  'auc': '0.9360 ± 0.0034',
  'kappa': '0.7570 ± 0.0155'}}

In [9]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['se'] # 'gated_attention', 'non_local', 'self_attention'
attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics

attention_results


Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with se attention...

Training run 1 with attention: se


2024-10-01 03:23:11.982681: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9796 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:c1:00.0, compute capability: 7.5


Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1727753010.054881 1470367 service.cc:146] XLA service 0x7fa9200019d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727753010.055013 1470367 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-10-01 03:23:30.519005: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-01 03:23:32.423968: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN versio

 2/26 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.4766 - loss: 0.7577  

...

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.4739 - loss: 0.7288 - val_accuracy: 0.5407 - val_loss: 0.6854
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.5139 - loss: 0.7065 - val_accuracy: 0.6870 - val_loss: 0.6453
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.6311 - loss: 0.6417 - val_accuracy: 0.7520 - val_loss: 0.5991
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.6865 - loss: 0.6039 - val_accuracy: 0.8089 - val_loss: 0.5487
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.7554 - loss: 0.5625 - val_accuracy: 0.8293 - val_loss: 0.4998
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - accuracy: 0.7638 - loss: 0.5260 - val_accuracy: 0.8333 - val_loss: 0.4640
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.7835 - loss: 0.4804 - val_accuracy: 0.8415 - val_loss: 0.4400
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.8287 - loss: 0.4433 - val_accuracy: 0.8374 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5858 - loss: 0.6569 - val_accuracy: 0.7236 - val_loss: 0.6000
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.6795 - loss: 0.6139 - val_accuracy: 0.7683 - val_loss: 0.5692
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.7090 - loss: 0.5904 - val_accuracy: 0.7886 - val_loss: 0.5356
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7008 - loss: 0.5814 - val_accuracy: 0.8089 - val_loss: 0.5048
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 294ms/step - accuracy: 0.7670 - loss: 0.5151 - val_accuracy: 0.8333 - val_loss: 0.4700
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.7832 - loss: 0.4989 - val_accuracy: 0.8374 - val_loss: 0.4425
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 300ms/step - accuracy: 0.7990 - loss: 0.4572 - val_accuracy: 0.8537 - val_loss: 0.4223
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.8167 - loss: 0.4401 - val_accuracy: 0.8577 -

{'se': {'accuracy': '0.8967 ± 0.0041',
  'precision': '0.9586 ± 0.0050',
  'recall': '0.8500 ± 0.0059',
  'f1_score': '0.9010 ± 0.0041',
  'auc': '0.9507 ± 0.0015',
  'kappa': '0.7939 ± 0.0082'}}

In [10]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['gc'] # 'gated_attention', 'non_local', 'self_attention'
attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics

attention_results


Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with gc attention...

Training run 1 with attention: gc
Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.6121 - loss: 5.4128 - val_accuracy: 0.7886 - val_loss: 1.7029
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7448 - loss: 3.1468 - val_accuracy: 0.8089 - val_loss: 1.2965
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.6746 - loss: 3.3267 - val_accuracy: 0.8537 - val_loss: 0.9986
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.7490 - loss: 2.3795 - val_accuracy: 0.8496 - val_loss: 0.9889
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7837 - loss: 1.7429 - val_accuracy: 0.8496 - val_loss: 0.9443
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 301ms/step - accuracy: 0.8005 - loss: 1.4818 - val_accuracy: 0.8699 - val_loss: 0.8127
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.7734 - loss: 1.5973 - val_accuracy: 0.8740 - val_loss: 0.8468
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.7665 - loss: 1.5769 - val_accuracy: 0.8618 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 945ms/step
Run 1 - Accuracy: 0.8537, Precision: 0.9167, Recall: 0.8088, F1 Score: 0.8594, AUC: 0.9422, Kappa: 0.7081

Training run 2 with attention: gc
Epoch 1/50


...

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 47s 986ms/step - accuracy: 0.6486 - loss: 4.6608 - val_accuracy: 0.7642 - val_loss: 1.9977
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.7703 - loss: 2.7484 - val_accuracy: 0.8049 - val_loss: 1.8320
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.7743 - loss: 2.2869 - val_accuracy: 0.8049 - val_loss: 1.1670
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 310ms/step - accuracy: 0.7577 - loss: 1.9899 - val_accuracy: 0.8333 - val_loss: 1.0641
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7751 - loss: 1.7631 - val_accuracy: 0.8415 - val_loss: 0.8856
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.7808 - loss: 1.4133 - val_accuracy: 0.8252 - val_loss: 0.9613
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 301ms/step - accuracy: 0.7996 - loss: 1.1816 - val_accuracy: 0.8496 - val_loss: 0.6830
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 308ms/step - accuracy: 0.8054 - loss: 0.9365 - val_accuracy: 0.853

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 800ms/step
Run 5 - Accuracy: 0.8089, Precision: 0.9320, Recall: 0.7059, F1 Score: 0.8033, AUC: 0.9158, Kappa: 0.6243
Accuracy - Mean: 0.8358, Std: 0.0156
Precision - Mean: 0.9244, Std: 0.0122
Recall - Mean: 0.7662, Std: 0.0396
F1_score - Mean: 0.8371, Std: 0.0201
Auc - Mean: 0.9199, Std: 0.0147
Kappa - Mean: 0.6746, Std: 0.0289


{'gc': {'accuracy': '0.8358 ± 0.0156',
  'precision': '0.9244 ± 0.0122',
  'recall': '0.7662 ± 0.0396',
  'f1_score': '0.8371 ± 0.0201',
  'auc': '0.9199 ± 0.0147',
  'kappa': '0.6746 ± 0.0289'}}

In [11]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['gated_attention'] # 'gated_attention', 'non_local', 'self_attention'
attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics

attention_results


Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with gated_attention attention...

Training run 1 with attention: gated_attention
Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.5537 - loss: 0.6817 - val_accuracy: 0.7927 - val_loss: 0.5709
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - accuracy: 0.7446 - loss: 0.5775 - val_accuracy: 0.8130 - val_loss: 0.5102
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.7769 - loss: 0.5202 - val_accuracy: 0.8252 - val_loss: 0.4685
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.7995 - loss: 0.4807 - val_accuracy: 0.8333 - val_loss: 0.4462
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.8078 - loss: 0.4518 - val_accuracy: 0.8293 - val_loss: 0.4249
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8199 - loss: 0.4504 - val_accuracy: 0.8415 - val_loss: 0.4107
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8347 - loss: 0.4377 - val_accuracy: 0.8415 - val_loss: 0.4005
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.8230 - loss: 0.4247 - val_accuracy: 0.8496 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 46s 962ms/step - accuracy: 0.6066 - loss: 0.6649 - val_accuracy: 0.7764 - val_loss: 0.5653
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.6963 - loss: 0.5839 - val_accuracy: 0.8171 - val_loss: 0.5043
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7672 - loss: 0.5247 - val_accuracy: 0.8293 - val_loss: 0.4646
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7726 - loss: 0.4965 - val_accuracy: 0.8455 - val_loss: 0.4406
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.8234 - loss: 0.4465 - val_accuracy: 0.8577 - val_loss: 0.4240
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 300ms/step - accuracy: 0.8408 - loss: 0.4499 - val_accuracy: 0.8618 - val_loss: 0.4078
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.8363 - loss: 0.4270 - val_accuracy: 0.8618 - val_loss: 0.3963
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8235 - loss: 0.4346 - val_accuracy: 0.869

...

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.5745 - loss: 0.6636 - val_accuracy: 0.7398 - val_loss: 0.5906
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7314 - loss: 0.5694 - val_accuracy: 0.8049 - val_loss: 0.5193
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7888 - loss: 0.5116 - val_accuracy: 0.8293 - val_loss: 0.4801
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.8094 - loss: 0.4843 - val_accuracy: 0.8252 - val_loss: 0.4556
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7931 - loss: 0.4744 - val_accuracy: 0.8415 - val_loss: 0.4320
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8073 - loss: 0.4812 - val_accuracy: 0.8537 - val_loss: 0.4150
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8000 - loss: 0.4608 - val_accuracy: 0.8537 - val_loss: 0.4035
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - accuracy: 0.8410 - loss: 0.3960 - val_accuracy: 0.8496 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 46s 964ms/step - accuracy: 0.5199 - loss: 0.7000 - val_accuracy: 0.7033 - val_loss: 0.6156
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.7039 - loss: 0.6027 - val_accuracy: 0.7967 - val_loss: 0.5341
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7787 - loss: 0.5284 - val_accuracy: 0.8211 - val_loss: 0.4885
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.7991 - loss: 0.4995 - val_accuracy: 0.8415 - val_loss: 0.4555
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 296ms/step - accuracy: 0.8306 - loss: 0.4547 - val_accuracy: 0.8577 - val_loss: 0.4342
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8288 - loss: 0.4368 - val_accuracy: 0.8618 - val_loss: 0.4230
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 295ms/step - accuracy: 0.8199 - loss: 0.4500 - val_accuracy: 0.8618 - val_loss: 0.4087
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 302ms/step - accuracy: 0.8200 - loss: 0.4479 - val_accuracy: 0.869

{'gated_attention': {'accuracy': '0.8894 ± 0.0054',
  'precision': '0.9489 ± 0.0090',
  'recall': '0.8456 ± 0.0066',
  'f1_score': '0.8943 ± 0.0050',
  'auc': '0.9475 ± 0.0015',
  'kappa': '0.7792 ± 0.0109'}}

In [10]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['non_local', 'self_attention'] 
attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics


Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with non_local attention...

Training run 1 with attention: non_local
Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1000ms/step - accuracy: 0.6652 - loss: 0.6927 - val_accuracy: 0.8415 - val_loss: 0.3779
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 300ms/step - accuracy: 0.8627 - loss: 0.3626 - val_accuracy: 0.8374 - val_loss: 0.4172
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8630 - loss: 0.3339 - val_accuracy: 0.8374 - val_loss: 0.3694
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 310ms/step - accuracy: 0.8701 - loss: 0.3267 - val_accuracy: 0.8943 - val_loss: 0.2961
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8719 - loss: 0.3147 - val_accuracy: 0.9024 - val_loss: 0.2949
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 298ms/step - accuracy: 0.8859 - loss: 0.2701 - val_accuracy: 0.8780 - val_loss: 0.3232
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.9125 - loss: 0.2160 - val_accuracy: 0.8821 - val_loss: 0.3383
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8893 - loss: 0.2434 - val_accuracy: 0.89

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 994ms/step - accuracy: 0.6465 - loss: 0.7905 - val_accuracy: 0.8699 - val_loss: 0.3615
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8089 - loss: 0.4223 - val_accuracy: 0.8659 - val_loss: 0.3734
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8856 - loss: 0.3416 - val_accuracy: 0.8780 - val_loss: 0.3374
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8586 - loss: 0.3875 - val_accuracy: 0.8984 - val_loss: 0.3100
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8754 - loss: 0.2972 - val_accuracy: 0.8984 - val_loss: 0.3190
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8836 - loss: 0.2947 - val_accuracy: 0.9024 - val_loss: 0.3377
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 301ms/step - accuracy: 0.9240 - loss: 0.2088 - val_accuracy: 0.9228 - val_loss: 0.3141
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.9081 - loss: 0.2566 - val_accuracy: 0.914

...

26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.6793 - loss: 0.6793 - val_accuracy: 0.8496 - val_loss: 0.4068
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.8441 - loss: 0.3961 - val_accuracy: 0.8537 - val_loss: 0.3611
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8681 - loss: 0.3373 - val_accuracy: 0.8740 - val_loss: 0.3361
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8841 - loss: 0.3378 - val_accuracy: 0.8780 - val_loss: 0.3475
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8740 - loss: 0.3162 - val_accuracy: 0.8862 - val_loss: 0.3216
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8813 - loss: 0.3164 - val_accuracy: 0.9024 - val_loss: 0.3160
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.8979 - loss: 0.2481 - val_accuracy: 0.9024 - val_loss: 0.3269
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.9077 - loss: 0.2495 - val_accuracy: 0.8943 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6993 - loss: 0.7191 - val_accuracy: 0.7927 - val_loss: 0.5327
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.7964 - loss: 0.4697 - val_accuracy: 0.8740 - val_loss: 0.3651
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8411 - loss: 0.4038 - val_accuracy: 0.8780 - val_loss: 0.3261
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8479 - loss: 0.3839 - val_accuracy: 0.9024 - val_loss: 0.3333
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8563 - loss: 0.3323 - val_accuracy: 0.9024 - val_loss: 0.2824
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 299ms/step - accuracy: 0.8758 - loss: 0.3013 - val_accuracy: 0.9065 - val_loss: 0.3039
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8938 - loss: 0.2647 - val_accuracy: 0.9106 - val_loss: 0.2974
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8943 - loss: 0.3044 - val_accuracy: 0.9065 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.5936 - loss: 0.8375 - val_accuracy: 0.7683 - val_loss: 0.5043
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.7791 - loss: 0.5046 - val_accuracy: 0.8374 - val_loss: 0.3920
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.8393 - loss: 0.3747 - val_accuracy: 0.8496 - val_loss: 0.3988
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8626 - loss: 0.3469 - val_accuracy: 0.8577 - val_loss: 0.3654
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8892 - loss: 0.2983 - val_accuracy: 0.8618 - val_loss: 0.3615
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8796 - loss: 0.3030 - val_accuracy: 0.8943 - val_loss: 0.3152
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8887 - loss: 0.3015 - val_accuracy: 0.8780 - val_loss: 0.3589
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 303ms/step - accuracy: 0.8954 - loss: 0.2675 - val_accuracy: 0.8862 -

In [11]:
attention_results

{'non_local': {'accuracy': '0.9073 ± 0.0157',
  'precision': '0.9396 ± 0.0079',
  'recall': '0.8897 ± 0.0312',
  'f1_score': '0.9136 ± 0.0161',
  'auc': '0.9565 ± 0.0066',
  'kappa': '0.8138 ± 0.0309'},
 'self_attention': {'accuracy': '0.9016 ± 0.0104',
  'precision': '0.9344 ± 0.0185',
  'recall': '0.8853 ± 0.0365',
  'f1_score': '0.9084 ± 0.0121',
  'auc': '0.9573 ± 0.0089',
  'kappa': '0.8023 ± 0.0198'}}

In [12]:
# Assuming EfficientNetB0 is your best-performing model
best_model_name = 'EfficientNetB0'
attention_types = ['gated_attention', 'non_local', 'self_attention'] 
# attention_results = {}

# Load your dataset
polyp_folder = '../colon_data/abnormal/'
non_polyp_folder = '../colon_data/normal/'
X, y = load_data(polyp_folder, non_polyp_folder)

# Evaluate models with different attention mechanisms
for attention_type in attention_types:
    print(f"\nEvaluating {best_model_name} with {attention_type} attention...")
    # metrics = evaluate_model_with_attention(X, y, best_model_name, attention_type)
    metrics = train_and_evaluate_model_with_attention(X_train, y_train, X_test, y_test, best_model_name, attention_type)
    attention_results[attention_type] = metrics


Total number of images: 1070, Image shape: (256, 256, 3)

Evaluating EfficientNetB0 with gated_attention attention...

Training model with attention: gated_attention
Epoch 1/50


/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.5209 - loss: 0.6941 - val_accuracy: 0.7317 - val_loss: 0.5845
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 315ms/step - accuracy: 0.7034 - loss: 0.5951 - val_accuracy: 0.7886 - val_loss: 0.5127
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 316ms/step - accuracy: 0.7878 - loss: 0.5411 - val_accuracy: 0.8130 - val_loss: 0.4735
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.7934 - loss: 0.4982 - val_accuracy: 0.8374 - val_loss: 0.4440
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.8073 - loss: 0.4737 - val_accuracy: 0.8455 - val_loss: 0.4284
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 316ms/step - accuracy: 0.8113 - loss: 0.4716 - val_accuracy: 0.8455 - val_loss: 0.4139
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.8237 - loss: 0.4297 - val_accuracy: 0.8455 - val_loss: 0.4017
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 312ms/step - accuracy: 0.8181 - loss: 0.4517 - val_accuracy: 0.8455 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.6584 - loss: 0.7205 - val_accuracy: 0.8577 - val_loss: 0.3891
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 313ms/step - accuracy: 0.8092 - loss: 0.4485 - val_accuracy: 0.8699 - val_loss: 0.3319
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8616 - loss: 0.3711 - val_accuracy: 0.8374 - val_loss: 0.3942
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.8492 - loss: 0.3422 - val_accuracy: 0.8699 - val_loss: 0.3090
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 316ms/step - accuracy: 0.9078 - loss: 0.2437 - val_accuracy: 0.8699 - val_loss: 0.3843
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 307ms/step - accuracy: 0.8760 - loss: 0.3134 - val_accuracy: 0.8699 - val_loss: 0.3113
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.8730 - loss: 0.2923 - val_accuracy: 0.9146 - val_loss: 0.2903
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 313ms/step - accuracy: 0.9116 - loss: 0.2328 - val_accuracy: 0.8943 -

/home/users/syellu1/.conda/envs/conda_env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6814 - loss: 0.6909 - val_accuracy: 0.8252 - val_loss: 0.3878
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 314ms/step - accuracy: 0.8340 - loss: 0.4267 - val_accuracy: 0.8537 - val_loss: 0.3753
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8336 - loss: 0.4278 - val_accuracy: 0.8659 - val_loss: 0.3426
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.8451 - loss: 0.3840 - val_accuracy: 0.8537 - val_loss: 0.4065
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 305ms/step - accuracy: 0.8729 - loss: 0.3187 - val_accuracy: 0.8659 - val_loss: 0.3193
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.9052 - loss: 0.2468 - val_accuracy: 0.8699 - val_loss: 0.3584
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 307ms/step - accuracy: 0.8938 - loss: 0.2866 - val_accuracy: 0.8984 - val_loss: 0.3155
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 311ms/step - accuracy: 0.8890 - loss: 0.2784 - val_accuracy: 0.8943 -

In [13]:
for attention_type, metrics in attention_results.items():
    print(f"\nAttention Mechanism: {attention_type}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print(f"Kappa: {metrics['kappa']:.4f}")


Attention Mechanism: spatial
Accuracy: 0.8902
Precision: 0.9098
Recall: 0.8897
F1 Score: 0.8996
AUC: 0.9342
Kappa: 0.7786

Attention Mechanism: se
Accuracy: 0.9024
Precision: 0.9516
Recall: 0.8676
F1 Score: 0.9077
AUC: 0.9507
Kappa: 0.8047

Attention Mechanism: gc
Accuracy: 0.8333
Precision: 0.9130
Recall: 0.7721
F1 Score: 0.8367
AUC: 0.9325
Kappa: 0.6689

Attention Mechanism: gated_attention
Accuracy: 0.8862
Precision: 0.9355
Recall: 0.8529
F1 Score: 0.8923
AUC: 0.9482
Kappa: 0.7722

Attention Mechanism: non_local
Accuracy: 0.9187
Precision: 0.9462
Recall: 0.9044
F1 Score: 0.9248
AUC: 0.9607
Kappa: 0.8364

Attention Mechanism: self_attention
Accuracy: 0.9390
Precision: 0.9764
Recall: 0.9118
F1 Score: 0.9430
AUC: 0.9733
Kappa: 0.8776


In [ ]:
for attention_type, metrics in attention_results.items():
    print(f"\nAttention Mechanism: {attention_type}")
    print(f"Average Accuracy: {np.mean(metrics['accuracy']):.4f}")
    print(f"Average Precision: {np.mean(metrics['precision']):.4f}")
    print(f"Average Recall: {np.mean(metrics['recall']):.4f}")
    print(f"Average F1 Score: {np.mean(metrics['f1_score']):.4f}")
    print(f"Average AUC: {np.mean(metrics['auc']):.4f}")
    print(f"Average Kappa: {np.mean(metrics['kappa']):.4f}")
